In [236]:
import geopandas as gpd
import pandas as pd
from datetime import datetime
from shapely.wkt import loads
from warnings import filterwarnings
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial import cKDTree

filterwarnings('ignore')

In this notebook. We'll be combining the fire data and the weather data into 1 data frame. The goals of this notebook is as follows:

- Find the closest 5 stations to a fire and create a dataframe based on that. We decided on 5 stations to create an average when modelling. 
- Connect the weather data to their respective stations based on Station ID and Date. 

Let's first grab the fire data below and check to make sure everything is prepped for combining

In [237]:
fire_data = gpd.read_file('Data/Fire_Data/fire_date_geo.shp',crs='esri:102009')


Let's also get the weather data:

In [238]:
monthly_weather= pd.read_csv('Data/Monthly_Weather_Data/monthly_weather.csv',index_col=0)

Since the weather data is in csv format, let's convert it to a geodataframe so that we can accurately determine distance between fire and weather station:

In [239]:
weather_shp=gpd.GeoDataFrame(monthly_weather,geometry=gpd.points_from_xy(monthly_weather['Longitude (x)'],monthly_weather['Latitude (y)']),crs=4326)

Let's now check the CRS for both the fire and Weather data. Remeber, a Coordinate reference system (CRS) defines, with the help of coordinates, how the two-dimensional, projected map is related to real locations on the earth:

In [240]:
fire_data['geometry'].crs

<Projected CRS: PROJCS["NAD_1983_Lambert_Conformal_Conic",GEOGCS[" ...>
Name: NAD_1983_Lambert_Conformal_Conic
Axis Info [cartesian]:
- [east]: Easting (metre)
- [north]: Northing (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: unnamed
- method: Lambert Conic Conformal (2SP)
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

Despite the `shp` file's CRS being set as 102009, it appears that the bounds are undefined. The bounds help us map the points based on a geographic space. If the bounds are not set, the distance between the points may be inaccurate. Let's reset the crs and ensure the bounds are defined:

In [241]:
fire_data=fire_data.set_crs('esri:102009',allow_override=True)

In [242]:
fire_data.crs

<Projected CRS: ESRI:102009>
Name: North_America_Lambert_Conformal_Conic
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: North America - onshore and offshore: Canada - Alberta; British Columbia; Manitoba; New Brunswick; Newfoundland and Labrador; Northwest Territories; Nova Scotia; Nunavut; Ontario; Prince Edward Island; Quebec; Saskatchewan; Yukon. United States (USA) - Alabama; Alaska (mainland); Arizona; Arkansas; California; Colorado; Connecticut; Delaware; Florida; Georgia; Idaho; Illinois; Indiana; Iowa; Kansas; Kentucky; Louisiana; Maine; Maryland; Massachusetts; Michigan; Minnesota; Mississippi; Missouri; Montana; Nebraska; Nevada; New Hampshire; New Jersey; New Mexico; New York; North Carolina; North Dakota; Ohio; Oklahoma; Oregon; Pennsylvania; Rhode Island; South Carolina; South Dakota; Tennessee; Texas; Utah; Vermont; Virginia; Washington; West Virginia; Wisconsin; Wyoming.
- bounds: (-172.54, 23.81, -47.74, 86.46)
Coordin

Now that the bounds are set, let's look at the weather data:

In [243]:
weather_shp.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

We notice that the CRS codes are different. We'll have to match them before we merge the tables. For now let's look at a sample of the data we have: 

In [244]:
fire_data.head()

,YEAR,MONTH,SRC_AGY2,geometry
0,2004,6,BC,"POLYGON Z ((-1886926.467 898021.006 0.000, -18..."
1,2004,6,BC,"POLYGON Z ((-1880308.251 892344.865 0.000, -18..."
2,2004,6,BC,"POLYGON Z ((-1965048.293 820512.199 0.000, -19..."
3,2004,6,BC,"POLYGON Z ((-1995073.527 854615.146 0.000, -19..."
4,2004,6,BC,"POLYGON Z ((-1988211.829 940418.674 0.000, -19..."


Let's now make a list of stations that we'll use to merge the stations and fire data by distance:

In [245]:
list_stations= weather_shp[['Climate ID','geometry']]

We have a list of all the stations in our dataframe, since it's listing weather information on a monthly bases, we will have multiple duplicates of our station numbers. Let's drop duplicates so we only have a list of unique `Climate ID`s.

In [246]:
list_stations.drop_duplicates(inplace=True,ignore_index=True)

In [247]:
weather_shp.drop_duplicates(inplace=True)

In [248]:
list_stations=gpd.GeoDataFrame(list_stations)

Let's now confirm that the crs is still the same for our new list:

In [249]:
list_stations.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [250]:
list_stations.shape

(430, 2)

In [251]:
list_stations

,Climate ID,geometry
0,709CEE9,POINT (-78.28000 48.80000)
1,704FEG0,POINT (-64.78000 50.28000)
2,702FR30,POINT (-71.73000 45.67000)
3,701LEEH,POINT (-72.40000 46.87000)
4,701HE63,POINT (-72.62000 46.38000)
...,...,...
425,1015105,POINT (-123.56000 48.37000)
426,1014820,POINT (-123.53000 48.57000)
427,1012710,POINT (-123.44000 48.43000)
428,1012055,POINT (-124.05000 48.83000)


Let's change the `fire_data` to match the coordinate system of our `list_stations` data before we start merging. 

In [252]:
fire_data=fire_data.to_crs(crs=4326)

### Using cKDTree to find the nearest fires to the respective stations.  

The cKDTree class provides an index into a set of k-dimensional points which can be used to rapidly look up the nearest neighbors of any point. this is the most effecient way of merging data based on distance.

Let's start with first extracting the coordinates.

In [253]:

# Extract coordinates
fire_coords = fire_data.geometry.apply(lambda geom: (geom.centroid.x, geom.centroid.y)).tolist()
station_coords = list_stations.geometry.apply(lambda geom: (geom.x, geom.y)).tolist()



Now we'll build the tree. We're going to use the station coordinates as a reference point, so that when we are looking at the fire coordinates, we are measuring the distance between them

In [254]:
# Build KDTree
tree = cKDTree(station_coords)


`.query()` is the function that will get us the distance between coordinates. We're collecting the distances and indicies to be able to create a new Dataframe with the connection. `k=5` means that we'll be looking at the top 5 nearest neighbours while `p=2` is referencing the use of the `Euclidean distance`.

In [255]:

# Query for nearest stations
distances, indices = tree.query(fire_coords, k=5,p=2)


In [256]:
distances

array([[2.80887921, 2.87810565, 2.88043013, 3.02735922, 3.04142161],
       [2.88839054, 2.96130313, 2.9684149 , 3.10751746, 3.12095583],
       [3.51319008, 3.5134173 , 3.55066469, 3.70396395, 3.71699966],
       ...,
       [1.34498919, 1.6041456 , 2.02475476, 2.27689879, 2.39921112],
       [1.08442195, 1.88584185, 2.23497577, 2.95078277, 3.01156395],
       [1.14448434, 1.9500969 , 2.30103576, 3.0323819 , 3.05382225]])

In [257]:
indices

array([[417, 427, 425,  29, 421],
       [417, 427, 425,  29, 421],
       [425, 417, 427, 426,  29],
       ...,
       [284, 288, 285, 328, 321],
       [320, 365, 367, 369, 285],
       [320, 365, 367, 369, 285]])

Let's create the dataframe with the stations and fires merged by distance:

In [258]:

# Construct the result dataframe
results = []
for i, fire in fire_data.iterrows():
    nearest_stations = list_stations.iloc[indices[i]].copy()
    nearest_stations['YEAR']=fire['YEAR']
    nearest_stations['MONTH']=fire['MONTH']
    nearest_stations['fire_index'] = i
    nearest_stations['distance'] = distances[i]
    results.append(nearest_stations)

result_df = pd.concat(results).reset_index(drop=True)

#merge with forest fires data
result_df = result_df.merge(fire_data[['geometry']], left_on='fire_index', right_index=True)

Let's confirm that the distances have been measured correctly. We can do that by confirming that a majority of our stations are assigned to atleast 1 fire occurance:

In [259]:
result_df['Climate ID'].value_counts()

Climate ID
5022125    4916
5021220    4815
5010140    4194
5022575    3755
6020559    3575
           ... 
7022494       1
7016840       1
7016800       1
7017585       1
704FEG0       1
Name: count, Length: 384, dtype: int64

Let's take a sample look at the full table

In [260]:
result_df

,Climate ID,geometry_x,YEAR,MONTH,fire_index,distance,geometry_y
0,1018611,POINT (-123.32000 48.41000),2004,6,0,2.808879,"POLYGON Z ((-122.16984 45.86643 0.00000, -122...."
1,1012710,POINT (-123.44000 48.43000),2004,6,0,2.878106,"POLYGON Z ((-122.16984 45.86643 0.00000, -122...."
2,1015105,POINT (-123.56000 48.37000),2004,6,0,2.880430,"POLYGON Z ((-122.16984 45.86643 0.00000, -122...."
3,1016RM0,POINT (-123.43000 48.60000),2004,6,0,3.027359,"POLYGON Z ((-122.16984 45.86643 0.00000, -122...."
4,1016940,POINT (-123.42000 48.62000),2004,6,0,3.041422,"POLYGON Z ((-122.16984 45.86643 0.00000, -122...."
...,...,...,...,...,...,...,...
134370,1181508,POINT (-121.63000 55.69000),1992,7,26874,1.144484,"MULTIPOLYGON Z (((-122.77133 55.84941 0.00000,..."
134371,1096468,POINT (-122.77000 53.88000),1992,7,26874,1.950097,"MULTIPOLYGON Z (((-122.77133 55.84941 0.00000,..."
134372,1093474,POINT (-122.70000 53.53000),1992,7,26874,2.301036,"MULTIPOLYGON Z (((-122.77133 55.84941 0.00000,..."
134373,1090660,POINT (-121.51000 53.07000),1992,7,26874,3.032382,"MULTIPOLYGON Z (((-122.77133 55.84941 0.00000,..."


In [261]:
#confirming there's no missing data
weather_shp.Date.isna().sum()

0

Let's change the `Date` column to datetime so that we can extract the month and year. 

In [262]:
weather_shp['Date']=pd.to_datetime(weather_shp['Date'])

In [263]:
#extract month and year by creating new columns

weather_shp['MONTH'] = weather_shp['Date'].dt.month
weather_shp['YEAR'] = weather_shp['Date'].dt.year

Now finally, we will merge the weather data based on `Month`, `YEAR`, and `Climate ID`. We are using an outer merge to capture all the stations that don't have any fires associated to them(i.e stations with data from winter months).

In [264]:
merged_df = pd.merge(result_df, weather_shp, how='outer', on=['MONTH','YEAR','Climate ID'])

In [265]:
merged_df.sample(20)

,Climate ID,geometry_x,YEAR,MONTH,fire_index,distance,geometry_y,Longitude (x),Latitude (y),Year,...,MT 1 month prior,TS 1 month prior,TP 1 month prior,MT 2 month prior,TS 2 month prior,TP 2 month prior,MT 3 month prior,TS 3 month prior,TP 3 month prior,geometry
104410,1145442,POINT (-117.21000 49.59000),2003,6,7949.0,0.699083,"POLYGON Z ((-117.20201 48.89107 0.00000, -117....",-117.21,49.59,2003.0,...,21.922581,0.000000,0.129032,21.025926,0.000000,0.529032,15.536667,0.000000,1.286667,POINT (-117.21000 49.59000)
13528,7020305,None,1991,2,NaN,NaN,None,-71.95,46.02,1991.0,...,-1.209677,1.354839,3.129032,6.810000,0.166667,3.283333,13.245161,0.000000,3.083871,POINT (-71.95000 46.02000)
89961,4015800,POINT (-102.12000 49.32000),2020,5,17020.0,5.438719,"MULTIPOLYGON Z (((-99.77295 44.41369 0.00000, ...",-102.12,49.32,2020.0,...,17.279310,0.000000,4.086207,19.430769,0.000000,1.746154,19.410000,0.000000,0.740000,POINT (-102.12000 49.32000)
245211,6070QK6,None,1992,11,NaN,NaN,None,-82.12,49.38,1992.0,...,-12.716129,2.400000,3.219355,-16.329032,1.367742,1.367742,-20.864286,0.600000,0.600000,POINT (-82.12000 49.38000)
132326,4014156,POINT (-105.25000 51.42000),2020,6,24772.0,1.300954,"POLYGON Z ((-106.24057 52.27389 0.00000, -106....",-105.25,51.42,2020.0,...,18.964516,NaN,0.900000,18.422581,NaN,0.229032,11.843333,NaN,0.693333,POINT (-105.25000 51.42000)
194419,6020559,POINT (-93.97000 48.63000),2006,8,11824.0,5.445953,"POLYGON Z ((-96.35665 43.73382 0.00000, -96.35...",-93.97,48.63,2006.0,...,13.270000,0.000000,1.086667,3.993548,0.800000,1.664516,-1.673333,0.326667,0.926667,POINT (-93.97000 48.63000)
208995,6020559,POINT (-93.97000 48.63000),2017,8,15716.0,6.124569,"POLYGON Z ((-97.04294 43.33277 0.00000, -97.04...",-93.97,48.63,2017.0,...,14.476667,0.000000,4.420000,7.409677,0.180645,1.567742,-4.553333,0.726667,0.726667,POINT (-93.97000 48.63000)
133688,1021480,None,1990,7,NaN,NaN,None,-125.45,50.33,1990.0,...,17.558065,0.000000,2.570968,14.103333,0.000000,0.520000,8.203226,0.000000,13.619355,POINT (-125.45000 50.33000)
16133,1097646,None,1998,2,NaN,NaN,None,-121.69,52.18,1998.0,...,0.329032,0.387097,1.154839,3.943333,0.646667,1.140000,11.154839,0.000000,0.987097,POINT (-121.69000 52.18000)
219979,1166658,None,2004,9,NaN,NaN,None,-120.80,50.94,2004.0,...,4.076923,0.806452,1.296774,-0.643333,1.100000,1.100000,-4.717857,1.564516,1.580645,POINT (-120.80000 50.94000)


In [266]:
merged_df.shape

(269906, 25)

In [267]:
merged_df.duplicated().sum()

0

In [268]:
merged_df.drop_duplicates(inplace=True)

Let's take a look at the missing data in our new completely merged dataframe:

In [269]:
for i in range(len(merged_df.columns)):
    na_index=merged_df.isna().sum().index[i]
    na_ratio=merged_df.isna().sum().iloc[i]
    print(f'{na_index} has {round((na_ratio/merged_df.shape[0])*100,2)}% missing data')


Climate ID has 0.0% missing data
geometry_x has 50.21% missing data
YEAR has 0.0% missing data
MONTH has 0.0% missing data
fire_index has 50.21% missing data
distance has 50.21% missing data
geometry_y has 50.21% missing data
Longitude (x) has 2.8% missing data
Latitude (y) has 2.8% missing data
Year has 2.8% missing data
Month has 2.8% missing data
Date has 2.8% missing data
Mean Temp (°C) has 13.04% missing data
Total Snow (cm) has 19.88% missing data
Total Precip (mm) has 6.52% missing data
MT 1 month prior has 13.22% missing data
TS 1 month prior has 20.07% missing data
TP 1 month prior has 6.61% missing data
MT 2 month prior has 13.39% missing data
TS 2 month prior has 20.24% missing data
TP 2 month prior has 6.84% missing data
MT 3 month prior has 13.51% missing data
TS 3 month prior has 20.43% missing data
TP 3 month prior has 7.34% missing data
geometry has 2.8% missing data


We can see that all the fire related datapoints have 51% missing data, which is expected as we have weather information on dates that didn't have any fires associated to them. These columns will eventually be dropped, as we only need to know whether or not a fire occured at that point.

In [270]:
merged_df.dtypes

Climate ID                   object
geometry_x                 geometry
YEAR                          int64
MONTH                         int64
fire_index                  float64
distance                    float64
geometry_y                 geometry
Longitude (x)               float64
Latitude (y)                float64
Year                        float64
Month                       float64
Date                 datetime64[ns]
Mean Temp (°C)              float64
Total Snow (cm)             float64
Total Precip (mm)           float64
MT 1 month prior            float64
TS 1 month prior            float64
TP 1 month prior            float64
MT 2 month prior            float64
TS 2 month prior            float64
TP 2 month prior            float64
MT 3 month prior            float64
TS 3 month prior            float64
TP 3 month prior            float64
geometry                   geometry
dtype: object

Below, I'll be creating a new column with a `1` label for cases where fire did occur and a `0` for when no fire occured. I'll be using `geometry` as the column to look at; if there's missing data, that means that no fire occured, otherwise fire did occur. This can be done with any column that came from the original fire dataframe. 

In [271]:
merged_df['Fire'] = np.where(merged_df['geometry_y']!=None,1,0)

In [272]:
merged_df.sample(10)

,Climate ID,geometry_x,YEAR,MONTH,fire_index,distance,geometry_y,Longitude (x),Latitude (y),Year,...,TS 1 month prior,TP 1 month prior,MT 2 month prior,TS 2 month prior,TP 2 month prior,MT 3 month prior,TS 3 month prior,TP 3 month prior,geometry,Fire
67839,1031316,None,2002,5,NaN,NaN,None,-125.22,48.79,2002.0,...,0.000000,3.550000,13.616129,0.000000,0.909677,13.325806,0.000000,0.174194,POINT (-125.22000 48.79000),0
47321,1023462,None,2006,4,NaN,NaN,None,-125.20,50.10,2006.0,...,0.000000,1.119355,16.453333,0.000000,0.446667,18.170370,0.000000,1.341935,POINT (-125.20000 50.10000),0
193114,6049095,POINT (-90.47000 49.03000),2005,8,17252.0,7.750081,"MULTIPOLYGON Z (((-91.13982 41.33057 0.00000, ...",-90.47,49.03,2005.0,...,NaN,5.593333,5.516129,NaN,3.225806,-4.976667,NaN,4.196667,POINT (-90.47000 49.03000),1
88178,7025670,POINT (-71.08000 45.40000),2018,5,18809.0,2.913752,"MULTIPOLYGON Z (((-69.76151 42.80102 0.00000, ...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,1
205306,1140876,POINT (-118.23000 49.02000),2015,8,3975.0,6.409118,"POLYGON Z ((-116.54194 42.83043 0.00000, -116....",-118.23,49.02,2015.0,...,0.000000,0.000000,10.903226,0.000000,1.432258,-0.082609,0.278261,1.852174,POINT (-118.23000 49.02000),1
165497,117CA90,POINT (-117.70000 51.24000),2012,7,2142.0,0.216654,"POLYGON Z ((-117.76233 51.47399 0.00000, -117....",-117.70,51.24,2012.0,...,NaN,NaN,NaN,NaN,NaN,-1.203333,4.233333,5.600000,POINT (-117.70000 51.24000),1
173115,5022575,POINT (-96.04000 49.35000),2016,7,15499.0,7.810487,"POLYGON Z ((-97.41559 41.66287 0.00000, -97.41...",-96.04,49.35,2016.0,...,0.000000,1.580645,NaN,0.000000,4.600000,NaN,0.000000,2.131034,POINT (-96.04000 49.35000),1
112700,4022359,POINT (-106.58000 51.13000),2008,6,9493.0,0.733290,"MULTIPOLYGON Z (((-105.82969 51.15052 0.00000,...",-106.58,51.13,2008.0,...,NaN,1.290323,18.432258,NaN,1.232258,12.070000,NaN,0.253333,POINT (-106.58000 51.13000),1
238583,1153034,POINT (-115.46000 49.52000),2010,10,5754.0,6.726840,"POLYGON Z ((-113.84345 42.99110 0.00000, -113....",-115.46,49.52,2010.0,...,1.105263,1.431579,-6.133333,0.888889,0.888889,-4.747059,1.450000,1.940000,POINT (-115.46000 49.52000),1
93183,7017585,None,1992,6,NaN,NaN,None,-72.43,46.53,1992.0,...,0.000000,6.316129,17.409677,0.000000,2.809677,13.296667,0.000000,3.320000,POINT (-72.43000 46.53000),0


Now we have a table showing whether fire had occured or not! 

Let's export the table as it is above, and we'll clean it up for modelling in the `Modelling.ipynb` notebook!

In [273]:
#exporting for modelling and analysis
merged_df.to_csv('Data/modelling_df.csv')


In [ ]:
predict= model.predict(dataframe_2022[:,df.columns != 'ratio'])
df_2023 = df_2022

df_2023['ratio'] = predict

df_2023['ratio_1'] =df_2023['ratio'].shift(1)
df_2023['ratio_2'] =df_2023['ratio'].shift(2)